# MIKAMI NCC

Program to graph Temperature (Inside,Outside, GDD) of Night Cooling Chamber (NCC) for Mikami Strawbery Farm. <br>
Sensor used are from [TANDD ](https://www.tandd.co.jp/product/rtr500_series.html).
Two sensors are installed inside and outside NCC.<br>

The temperature readings can be viewed in this [link ](http://nodai.net/mikami/ncc.php)<br>

The plots are made using [Bokeh ](http://bokeh.pydata.org/en/latest/) a Python interactive visualization library



In [1]:
#Load modules 
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
import sys 

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bkcharts import * 
from bokeh.palettes import PuBu

from bokeh.models.widgets import Tabs, Panel
import pandas, datetime
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from time import gmtime,strftime
from calendar import timegm
from bokeh.layouts import row, column

from bokeh.plotting import figure
from bokeh.models import Range1d
from bokeh.embed import components

from bokeh.models import ColumnDataSource,HoverTool
from time import gmtime,strftime
from calendar import timegm

# Connection to MySQL Database to get the data
#conn = pymysql.connect(host = 'localhost', port= 8888, user ='root', passwd = 'root', db = 'test_DB')
conn = pymysql.connect(host = '', port= , user ='', passwd = '', db = '')

cursor1 = conn.cursor()
cursor2 = conn.cursor()
cursor3 = conn.cursor()

cursor1.execute('SELECT date_time,measure FROM tandd WHERE name = "cool" ORDER BY date_time ASC')
cursor2.execute('SELECT date_time,measure FROM tandd WHERE name = "house" ORDER BY date_time ASC')
cursor3.execute('SELECT date_time,ROUND((MAX(measure)+MIN(measure))/2-10,1) from tandd group by DAY(date_time) ORDER BY date_time ASC')

result1 = cursor1.fetchall()
result2 = cursor2.fetchall()
result3 = cursor3.fetchall()

df1 = pd.DataFrame( [[ij for ij in i] for i in result1] )
df2 = pd.DataFrame( [[ij for ij in i] for i in result2] )
df3 = pd.DataFrame( [[ij for ij in i] for i in result3] )

df1.rename(columns={0: 'Date', 1: 'TEMP_IN'}, inplace=True);
df2.rename(columns={0: 'Date', 1: 'TEMP_OUT'}, inplace=True);
df3.rename(columns={0: 'Date', 1:'GDD'}, inplace=True);
df3['Cum_GDD'] = df3.GDD.cumsum()

if cursor1 and conn:
    cursor1.close()
if cursor2 and conn:
    cursor2.close()
if cursor3 and conn:
    cursor3.close()
    conn.close()
    
#outputs in Jupyter    
output_notebook()
#Create the source data with formats for date
Date1 = df1.Date
Date2 = df2.Date
Date3 = df3.Date
Temp_In = df1.TEMP_IN
Temp_Out = df2.TEMP_OUT
GDD = df3.GDD
Cum_GDD = df3.Cum_GDD
source = ColumnDataSource(
    data=dict(
        Temp_In = Temp_In,
        Temp_Out = Temp_Out,
        GDD = GDD,
        Cum_GDD = Cum_GDD,
        Date1 = pd.to_datetime(df1['Date'], format='%Y-%m-%d'),
        Date2 = pd.to_datetime(df2['Date'], format='%Y-%m-%d'),
        Date3 = pd.to_datetime(df3['Date'], format='%Y-%m-%d')
    )
)
#Hover with specfied tooltips
hover1 = HoverTool(
    tooltips="""
         <div style="background: #FFFFFF;">温度
            <span style="font-size: 18px; color: blue;"></span><br />
            <span style="font-size: 16px; color: green;">( @Temp_In )</span>
        </div>
    """)
hover2 = HoverTool(
    tooltips="""
         <div style="background: #FFFFFF;">
            <span style="font-size: 18px; color: blue;">温度</span><br />
            <span style="font-size: 16px; color: green;">( @Temp_Out )</span>
        </div>
    """)
hover3 = HoverTool(
    tooltips="""
         <div style="background: #FFFFFF;">
            <span style="font-size: 18px; color: blue;">GDD</span><br />
            <span style="font-size: 16px; color: green;">( @GDD )</span>
        </div>
    """)
hover4 = HoverTool(
    tooltips="""
         <div style="background: #FFFFFF;">
            <span style="font-size: 18px; color: blue;">積算温度</span><br />
            <span style="font-size: 16px; color: green;">( @Cum_GDD )</span>
        </div>
    """)

# Create figures
p4 = figure(x_axis_label='日付',x_axis_type='datetime',
            y_axis_label='温度 (℃)',
            background_fill_color='#BFE6F6',
            tools=[hover1, 'xpan','reset','previewsave'],
            plot_width=1000,
            #border_fill_color = '#5a9999',
            logo=None)
p4.line(x='Date1', y='Temp_In', legend="温度",source=source,line_width=1,line_color="blue")
p4.xaxis.axis_label_text_font_size = "20pt"
p4.yaxis.axis_label_text_font_size = "20pt"

p5 = figure(x_axis_label='日付',x_axis_type='datetime',
            y_axis_label='温度 (℃)',
            background_fill_color='#BFE6F6',
            tools=[hover2, 'xpan', 'reset','previewsave'],
            plot_width=1000,
            logo=None)
p5.line(x='Date2', y='Temp_Out', legend="温度",source=source,line_width=1,line_color="blue")
p5.xaxis.axis_label_text_font_size = "20pt"
p5.yaxis.axis_label_text_font_size = "20pt"

p6 = figure(x_axis_label='日付', x_axis_type='datetime',
            y_axis_label='温度 (℃)',
            background_fill_color='#BFE6F6',
            tools=[hover3, 'xpan','reset','previewsave'],
            plot_width=1000,
            logo=None)
p6.line(x='Date3', y='GDD',source=source,line_width=1,line_color="blue")
p6.circle('Date3','GDD',source=source)
p6.xaxis.axis_label_text_font_size = "20pt"
p6.yaxis.axis_label_text_font_size = "20pt"


p7 = figure(x_axis_label='日付', x_axis_type='datetime',
            y_axis_label='積算温度',
            background_fill_color='#BFE6F6',
            tools=[hover4, 'xpan','reset','previewsave'],
            plot_width=1000,
            logo=None)
p7.line(x='Date3', y='Cum_GDD',source=source,line_width=1,line_color="blue")
p7.circle('Date3','Cum_GDD',source=source)
p7.xaxis.axis_label_text_font_size = "20pt"
p7.yaxis.axis_label_text_font_size = "20pt"

# Create the Tabs of the figures and plots
tab4 = Panel(child=p4, title='中側')
tab5 = Panel(child=p5, title='外側')
tab6 = Panel(child=p6, title='GDD')
tab7 = Panel(child=p7, title='積算温度')

layout = Tabs(tabs=[tab4, tab5, tab6, tab7])
output_file('MIKAMI_NCC.html')
show(layout)





Loading BokehJS ...

/Users/denisrubanga/anaconda/lib/python2.7/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('Cum_GDD', 14), ('Date1', 1227), ('Date2', 1237), ('Date3', 14), ('GDD', 14), ('Temp_In', 1227), ('Temp_Out', 1237)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
